In [10]:
# Parameters
batch_size = 2
seq_length = 5
rnn_size   = 10
vocab_size = 3

In [19]:
# RNN Sampler
import tensorflow as tf
from tensorflow.contrib.distributions import Categorical
tf.reset_default_graph()


def generate():
    # Initial indices
    indices = tf.constant(0, shape=[batch_size])

    # Embeddings and Logits
    embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
    softmax   = tf.get_variable('softmax', [rnn_size, vocab_size])

    # RNN
    outputs = []
    cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
    state = cell.zero_state(batch_size, tf.float32)
    inp = tf.nn.embedding_lookup(embedding, indices)

    with tf.variable_scope('rnn'):
        for i in xrange(seq_length):
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            rnn_out, state = cell(inp, state)
            logits_out = tf.matmul(rnn_out, softmax)
            outputs.append(logits_out)
            indices = tf.squeeze(Categorical(logits_out).sample(n=1))
    return outputs
            

with tf.Session() as sess:
    outputs = generate()
    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    for i, output in enumerate(outputs):
        print 'Iteration %d'%i
        print sess.run(output),'\n'

Iteration 0
[[ 0.49059618  0.09215772 -0.33202538]
 [ 0.49059618  0.09215772 -0.33202538]] 

Iteration 1
[[ 0.14282845  0.25214738 -0.47134605]
 [ 0.14282845  0.25214738 -0.47134605]] 

Iteration 2
[[ 0.0134341   0.33580467 -0.3240346 ]
 [ 0.0134341   0.33580467 -0.3240346 ]] 

Iteration 3
[[-0.14349148  0.34163788 -0.24653435]
 [-0.14349148  0.34163788 -0.24653435]] 

Iteration 4
[[-0.14806585  0.37420693 -0.21538797]
 [-0.14806585  0.37420693 -0.21538797]] 

